## 영상불러와서 OD

In [12]:
import cv2
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def detect_objects_on_video(video_path, model, threshold=0.5):
    # 비디오를 열고 비디오의 프레임을 처리합니다.
    cap = cv2.VideoCapture(video_path)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 객체 검출 실행
detect_objects_on_video('path_to_your_video.mp4', model)


## 내장카메라 with cv2 & resnet50

In [6]:
# COCO DATASET
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

## 사람만

In [13]:
import cv2
import torch
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def detect_objects_on_webcam(model, threshold=0.5):
    # 웹캠으로부터 비디오를 캡처합니다.
    cap = cv2.VideoCapture(0)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                # 'person' 클래스의 인덱스는 1입니다.
                if prediction[0]['scores'][i] > threshold and prediction[0]['labels'][i] == 1:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 웹캠에서 객체 검출을 실행
detect_objects_on_webcam(model)

## 성능

In [7]:
# 누적
def detect_objects_on_webcam(model, threshold=0.5):
    # 웹캠으로부터 비디오를 캡처합니다.
    cap = cv2.VideoCapture(0)

    # 특정 클래스를 추적하기 위한 카운터를 초기화합니다.
    class_counter = 0

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.

            # Only Person
            # tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            # if torch.cuda.is_available():
            #     tensor_image = tensor_image.cuda()

            # with torch.no_grad():
            #     # 모델 예측
            #     prediction = model(tensor_image)

            # # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # # 예를 들어, bounding box와 함께 객체를 그립니다.
            # for i in range(prediction[0]['boxes'].shape[0]):
            #     # 'person' 클래스의 인덱스는 1입니다.
            #     if prediction[0]['scores'][i] > threshold and prediction[0]['labels'][i] == 1:
            #         box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
            #         cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
            #         class_counter += 1

            # # 카운터 값을 화면에 표시합니다.
            # cv2.putText(frame, f"Detected {class_counter} objects", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)



            # 모든 사물
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    label = COCO_INSTANCE_CATEGORY_NAMES[prediction[0]['labels'][i]]
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
                    cv2.putText(frame, label, (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)


            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 웹캠에서 객체 검출을 실행
detect_objects_on_webcam(model)

: 

In [4]:
def detect_objects_on_webcam(model, threshold=0.5):
    # 웹캠으로부터 비디오를 캡처합니다.
    cap = cv2.VideoCapture(0)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                # 'person' 클래스의 인덱스는 1입니다.
                if prediction[0]['scores'][i] > threshold and prediction[0]['labels'][i] == 1:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
                    score = prediction[0]['scores'][i].item()

                    # 점수(또는 confidence) 값을 화면에 표시합니다.
                    cv2.putText(frame, f"Confidence: {score:.2f}", (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 웹캠에서 객체 검출을 실행
detect_objects_on_webcam(model)

## 원본

In [3]:
import cv2
import torch
from PIL import Image
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F

def detect_objects_on_webcam(model, threshold=0.5):
    # 웹캠으로부터 비디오를 캡처합니다.
    cap = cv2.VideoCapture(0)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 웹캠에서 객체 검출을 실행
detect_objects_on_webcam(model)

c:\Users\user\miniconda3\envs\nic\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\miniconda3\envs\nic\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
import time

def detect_objects_on_webcam(model, threshold=0.5):
    # 웹캠으로부터 비디오를 캡처합니다.
    cap = cv2.VideoCapture(0)

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            start_time = time.time()

            # OpenCV는 BGR을 사용하지만, PyTorch는 RGB를 사용하므로 색상을 변환합니다.
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # 이미지를 PIL 이미지로 변환합니다.
            pil_image = Image.fromarray(frame)
            # 모델을 호출하기 전에 이미지를 텐서로 변환합니다.
            tensor_image = F.to_tensor(pil_image).unsqueeze(0)

            if torch.cuda.is_available():
                tensor_image = tensor_image.cuda()

            with torch.no_grad():
                # 모델 예측
                prediction = model(tensor_image)

            # 이제 프레임에 객체 검출을 시각화할 수 있습니다.
            # 예를 들어, bounding box와 함께 객체를 그립니다.
            for i in range(prediction[0]['boxes'].shape[0]):
                if prediction[0]['scores'][i] > threshold:
                    box = prediction[0]['boxes'][i].cpu().numpy().astype(int)
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            end_time = time.time()
            fps = 1 / (end_time - start_time)
            print("FPS: ", fps)

            # 결과를 보여줍니다.
            cv2.imshow('frame', frame)

            # 'q' 키를 누르면 종료합니다.
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# 모델 로드
model = fasterrcnn_resnet50_fpn(pretrained=True)

if torch.cuda.is_available():
    model = model.cuda()

model.eval()

# 웹캠에서 객체 검출을 실행
detect_objects_on_webcam(model)

NameError: name 'fasterrcnn_resnet50_fpn' is not defined

## Pytorch with YOLO
pip install -U torch torchvision cython
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
git clone https://github.com/ultralytics/yolov5  # YOLOv5 라이브러리 복사
cd yolov5
pip install -r requirements.txt  # 종속성 설치

In [ ]:
# !pip install -U torch torchvision cython
# !pip install -U git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI
# !git clone https://github.com/ultralytics/yolov5
# %cd C:\Users\user\Desktop\project\model\yolov5
# !pip install -r requirements.txt
# !pip install --upgrade yolov5

In [1]:
import sys
print(sys.path)

['c:\\Users\\user\\Desktop\\project', 'c:\\Users\\user\\miniconda3\\envs\\nic\\python39.zip', 'c:\\Users\\user\\miniconda3\\envs\\nic\\DLLs', 'c:\\Users\\user\\miniconda3\\envs\\nic\\lib', 'c:\\Users\\user\\miniconda3\\envs\\nic', '', 'C:\\Users\\user\\AppData\\Roaming\\Python\\Python39\\site-packages', 'c:\\Users\\user\\miniconda3\\envs\\nic\\lib\\site-packages', 'c:\\Users\\user\\miniconda3\\envs\\nic\\lib\\site-packages\\win32', 'c:\\Users\\user\\miniconda3\\envs\\nic\\lib\\site-packages\\win32\\lib', 'c:\\Users\\user\\miniconda3\\envs\\nic\\lib\\site-packages\\Pythonwin']


In [1]:
%cd yolov5

c:\Users\user\Desktop\project\model\yolov5


In [2]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\user/.cache\torch\hub\master.zip
WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5  2023-7-17 Python-3.9.17 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [5]:
# !pip uninstall utils
!pip install utils

In [ ]:
from PIL import Image
import cv2
import numpy as np

from yolov5.models.common import DetectMultiBackend

from yolov5.models.experimental import attempt_load
from yolov5.utils.dataloaders import LoadStreams, LoadImages
from yolov5.utils.general import check_img_size, non_max_suppression#, scale_coords
from yolov5.utils.torch_utils import select_device #, load_classifier, time_synchronized

def detect(save_img=False):
    source, weights, view_img, imgsz = '0', 'yolov5s.pt', True, 640  # 웹캠을 소스로 사용하려면 '0'을 사용합니다.

    # Initialize
    device = select_device('')  # GPU가 있다면 사용합니다.
    model = attempt_load(weights, map_location=device)  # 모델을 로드합니다.
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # img_size 검사


    # Webcam에서 영상을 가져옵니다.
    dataset = LoadStreams(source, img_size=imgsz, stride=stride)

    names = model.module.names if hasattr(model, 'module') else model.names  # 클래스 이름 가져오기
    colors = [[np.random.randint(0, 255) for _ in range(3)] for _ in names]  # 클래스별 색상 설정

    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # FP32 모델을 실행합니다.

    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.float()  # uint8 to fp32
        img /= 255.0  # (0 - 255) to (0.0 - 1.0)

        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        pred = model(img, augment=False)[0]

        # Apply NMS
        pred = non_max_suppression(pred, 0.4, 0.5)

        for i, det in enumerate(pred):  # detections per image
            im0 = im0s[i].copy()

            # 테스트 결과를 보여줍니다.
            for *xyxy, conf, cls in reversed(det):
                label = f'{names[int(cls)]} {conf:.2f}'
                plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

            cv2.imshow(str(source), im0)
            if cv2.waitKey(1) == ord('q'):  # 'q'를 누르면 종료
                break

    print('Done.')
    cv2.destroyAllWindows()

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

detect()

## 결과 : 실패
 - 원인 : yolov5.utils.datasets 설치 오류로 인해 실패
 - 시간이 짧아 원인분석 후 문제해결하기 어려우므로 다른 모델을 사용하도록 하겠음

## yolov4
yolov4.weights: https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights
yolov4.cfg: https://github.com/AlexeyAB/darknet/blob/master/cfg/yolov4.cfg
coco.names: https://github.com/AlexeyAB/darknet/blob/master/data/coco.names

In [ ]:
import cv2
import numpy as np

# YOLOv4에서 사용하는 클래스 이름을 불러옵니다.
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# YOLOv4 네트워크를 로드합니다.
net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')

# 웹캠 비디오 스트림을 시작합니다.
cap = cv2.VideoCapture(0)

while True:
    ret, img = cap.read()
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)

    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    boxes = []
    confidences = []
    class_ids = []

    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(img, label, (x, y+30), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 2)

    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 얼굴인식 with mediapipe / OPERATING OK

In [20]:
# !pip install --upgrade protobuf
# !pip install --upgrade mediapipe

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\user\\miniconda3\\envs\\nic\\Lib\\site-packages\\google\\~upb\\_message.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



     -------------------------------------- 904.2/904.2 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Uninstalling protobuf-4.23.4:
      Successfully uninstalled protobuf-4.23.4


In [23]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection() as face_detection:
  while cap.isOpened():
    success, image = cap.read()
    while not success:
      success, image = cap.read()
    
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image)

    # Draw the face detection annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if results.detections:
      for detection in results.detections:
        mp_drawing.draw_detection(image, detection)
        
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Detection', cv2.flip(image, 1))
    
    # Exit on esc
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## yolov3

In [ ]:
import cv2
import numpy as np

# YOLO 네트워크 불러오기
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# 웹캠에서 스트림을 열기
cap = cv2.VideoCapture(0)

while True:
    # 웹캠으로부터 이미지를 읽어오기
    ret, img = cap.read()

    if not ret:
        break

    height, width, channels = img.shape

    # YOLO에서 사용하는 형태로 이미지 변환
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # 객체 검출
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 위치 조정
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # 노이즈 제거
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 2)
            cv2.putText(img, label, (x, y + 30), font, 2, (0,255,0), 3)

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()